In [35]:
import sagemaker
import boto3

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

print(f"sagemaker role arn: {role}")

sagemaker role arn: arn:aws:iam::334586217754:role/service-role/AmazonSageMaker-ExecutionRole-20240112T140048


In [36]:
from sagemaker.huggingface import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data="s3://sang-sagemaker-practice/my_model.tar.gz",  # path to your trained sagemaker model
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.26", # transformers version used
   pytorch_version="1.13", # pytorch version used
   py_version="py39", # python version of the DLC
)

In [37]:
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

-----!

In [38]:
# def func(stream, content_type):
#     return stream
predictor.serializer= sagemaker.base_serializers.JSONSerializer()
predictor.deserializer = sagemaker.base_deserializers.JSONDeserializer()

In [39]:
data = {
    "my_input": """
James Lovelock, Whose Gaia Theory Saw the Earth as Alive, Dies at 103
A British ecologist, he captured imaginations with his hypothesis and was essential to today’s understanding of man-made pollutants and their effect on the climate.

Share full article


99

James Lovelock in 1962. Among his inventions was the Electron Capture Detector, an inexpensive, portable, exquisitely sensitive device used to measure the spread of toxic man-made compounds in the environment.Credit...Donald Uhrbrock/Getty Images

By Keith Schneider
Published July 27, 2022
Updated Aug. 11, 2022
James Lovelock, the maverick British ecologist whose work was essential to today’s understanding of man-made pollutants and their effect on climate and who captured the scientific world’s imagination with his Gaia theory, portraying the Earth as a living creature, died on Tuesday, his 103rd birthday, at his home in Dorset, in southwest England.

His family confirmed the death in a statement on Twitter, saying that until six months ago he “was still able to walk along the coast near his home in Dorset and take part in interviews, but his health deteriorated after a bad fall earlier this year.”

Dr. Lovelock’s breadth of knowledge extended from astronomy to zoology. In his later years he became an eminent proponent of nuclear power as a means to help solve global climate change and a pessimist about humankind’s capacity to survive a rapidly warming planet.

But his global renown rested on three main contributions that he developed during a particularly abundant decade of scientific exploration and curiosity stretching from the late 1950s through the last half of the ’60s.

One was his invention of the Electron Capture Detector, an inexpensive, portable, exquisitely sensitive device used to help measure the spread of toxic man-made compounds in the environment. The device provided the scientific foundations of Rachel Carson’s 1962 book, “Silent Spring,” a catalyst of the environmental movement.

The detector also helped provide the basis for regulations in the United States and in other nations that banned harmful chemicals like DDT and PCBs and that sharply reduced the use of hundreds of other compounds as well as the public’s exposure to them.

Later, his finding that chlorofluorocarbons — the compounds that powered aerosol cans and were used to cool refrigerators and air-conditioners — were present in measurable concentrations in the atmosphere led to the discovery of the hole in the ozone layer. (Chlorofluorocarbons are now banned in most countries under a 1987 international agreement.)

But Dr. Lovelock may be most widely known for his Gaia theory — that Earth functioned, as he put it, as a “living organism” that is able to “regulate its temperature and chemistry at a comfortable steady state.”

The seeds of the idea were planted in 1965, when he was a member of the space exploration team recruited by the National Aeronautics and Space Administration and stationed at NASA’s Jet Propulsion Laboratory in Pasadena, Calif.

Editors’ Picks

Breathing Their Way to an Altered State

Unstoppable in Red

The Tech That Needs Fixing in 2024, and What Got Fixed Last Year
As an expert on the chemical composition of the atmospheres of Earth and Mars, Dr. Lovelock wondered why Earth’s atmosphere was so stable. He theorized that something must be regulating heat, oxygen, nitrogen and other components.

Climate Forward  There’s an ongoing crisis — and tons of news. Our newsletter keeps you up to date. Get it with a Times subscription.
“Life at the surface must be doing the regulation,” he later wrote.

He presented the theory in 1967 at a meeting of the American Astronautical Society in Lansing, Mich., and in 1968 at a scientific gathering at Princeton University.
"""
}


In [31]:
import json
t = predictor.serializer.serialize(data)
json.loads(t)

{'my_input': '\nJames Lovelock, Whose Gaia Theory Saw the Earth as Alive, Dies at 103\nA British ecologist, he captured imaginations with his hypothesis and was essential to today’s understanding of man-made pollutants and their effect on the climate.\n\nShare full article\n\n\n99\n\nJames Lovelock in 1962. Among his inventions was the Electron Capture Detector, an inexpensive, portable, exquisitely sensitive device used to measure the spread of toxic man-made compounds in the environment.Credit...Donald Uhrbrock/Getty Images\n\nBy Keith Schneider\nPublished July 27, 2022\nUpdated Aug. 11, 2022\nJames Lovelock, the maverick British ecologist whose work was essential to today’s understanding of man-made pollutants and their effect on climate and who captured the scientific world’s imagination with his Gaia theory, portraying the Earth as a living creature, died on Tuesday, his 103rd birthday, at his home in Dorset, in southwest England.\n\nHis family confirmed the death in a statement o

In [40]:
# request
predictor.predict(data)

{'output': '</s><s>James Lovelock, Whose Gaia Theory Saw the Earth as Alive, Dies at 103. A British ecologist, he captured imaginations with his hypothesis and was essential to today’s understanding of man-made pollutants and their effect on the climate. Among his inventions was the Electron Capture Detector, an inexpensive, portable, exquisitely sensitive device used to measure the spread of toxic man- made compounds in the environment. His family confirmed the death in a statement on Twitter, saying that until six months ago he was still able to walk along the coast near his home in Dorset.</s>'}

In [41]:
predictor.delete_model()
predictor.delete_endpoint()